# Python Implementation of the Schnorr Identification Protocol

This notebook presents a detailed Python implementation of the Schnorr Identification Protocol, a fundamental zero-knowledge proof system in cryptography. It serves as a practical complement to the theoretical discussion provided in [Intro to Schnorr](./schnorr_1.ipynb), where the mathematical foundations of the protocol are explored.

---

## Setup

The setup involves defining functions for parameter generation and key creation. These functions are implemented manually in this notebook to provide a clear understanding of the protocol's inner workings.

In [30]:
import random

def generate_parameters():
    while True:
        # Generate a random prime p
        p = random.randint(20, 100)
        if all(p % i != 0 for i in range(2, int(p**0.5) + 1)):
            # Find a prime q such that q divides p-1
            for q in range(2, p):
                if (p - 1) % q == 0 and all(q % i != 0 for i in range(2, int(q**0.5) + 1)):
                    # Find a generator g for the subgroup of order q
                    for g in range(2, p):
                        if pow(g, q, p) == 1 and pow(g, (p-1)//q, p) != 1:
                            return p, q, g

def keygen(p, q, g):
    x = random.randint(1, q-1)  # Prover's secret
    y = pow(g, x, p)            # Public key
    return x, y

## Protocol Execution

The following code snippets illustrate the process of proving the Schnorr Identification Protocol, the response and the verification steps.

In [31]:
def schnorr_prove(p, q, g, x):
    r = random.randint(0, q-1)
    t = pow(g, r, p)
    return r, t

def schnorr_response(p, q, g, x, r, c):
    s = (r + c * x) % q
    return s

def schnorr_verify(p, q, g, y, t, c, s):
    lhs = pow(g, s, p)
    rhs = (t * pow(y, c, p)) % p
    return lhs == rhs

## Worked Example

Let's work through a full protocol instance using small primes for easy verification.

In [35]:
# Generate small parameters for demonstration
p, q, g = generate_parameters()
print(f"parameters generated: p = {p}, q = {q}, g = {g}")
x, y = keygen(p, q, g)

print("=== Schnorr Identification Protocol ===")
print(f"Public parameters: p = {p}, q = {q}, g = {g}")
print(f"Prover's secret: x = {x}")
print(f"Prover's public key: y = {y}")
print()

parameters generated: p = 89, q = 11, g = 2
=== Schnorr Identification Protocol ===
Public parameters: p = 89, q = 11, g = 2
Prover's secret: x = 9
Prover's public key: y = 67



### Example 1: Single protocol execution

The honest prover successfully demonstrates knowledge of the secret `x`. The verifier accepts the proof, confirming the protocol's correctness.

In [36]:
print("=== Example 1: Single Protocol Execution ===")
# Prover computes commitment
r, t = schnorr_prove(p, q, g, x)
print(f"Prover selects random r = {r} and computes commitment t = {t}")

# Verifier sends challenge
c = random.randint(0, q-1)
print(f"Verifier selects random challenge c = {c}")

# Prover computes response
s = schnorr_response(p, q, g, x, r, c)
print(f"Prover computes response s = {s}")

# Verifier checks
result = schnorr_verify(p, q, g, y, t, c, s)
print(f"Verification result: {result}")
print()

=== Example 1: Single Protocol Execution ===
Prover selects random r = 0 and computes commitment t = 1
Verifier selects random challenge c = 2
Prover computes response s = 7
Verification result: True



### Example 2: Multiple protocol executions with different challenges

The protocol is executed multiple times with different challenges. Each execution demonstrates the repeatability and robustness of the protocol.

In [37]:
print("=== Example 2: Multiple Protocol Executions ===")
for i in range(3):
    print(f"--- Execution {i+1} ---")
    # Prover computes commitment
    r, t = schnorr_prove(p, q, g, x)
    print(f"Prover selects random r = {r} and computes commitment t = {t}")

    # Verifier sends challenge
    c = random.randint(0, q-1)
    print(f"Verifier selects random challenge c = {c}")

    # Prover computes response
    s = schnorr_response(p, q, g, x, r, c)
    print(f"Prover computes response s = {s}")

    # Verifier checks
    result = schnorr_verify(p, q, g, y, t, c, s)
    print(f"Verification result: {result}")
    print()

=== Example 2: Multiple Protocol Executions ===
--- Execution 1 ---
Prover selects random r = 9 and computes commitment t = 67
Verifier selects random challenge c = 8
Prover computes response s = 4
Verification result: True

--- Execution 2 ---
Prover selects random r = 9 and computes commitment t = 67
Verifier selects random challenge c = 0
Prover computes response s = 9
Verification result: True

--- Execution 3 ---
Prover selects random r = 5 and computes commitment t = 32
Verifier selects random challenge c = 8
Prover computes response s = 0
Verification result: True



### Example 3: Simulating a cheating prover

The cheating prover attempts to convince the verifier without knowing the secret. The verifier rejects the proof, demonstrating the protocol's soundness.

In [41]:
print("=== Example 3: Simulating a Cheating Prover ===")
# Cheating prover does not know x but tries to fake a response
fake_r = random.randint(0, q-1)
fake_t = pow(g, fake_r, p)
print(f"Cheating prover selects random fake_r = {fake_r} and computes fake_t = {fake_t}")

# Verifier sends challenge
c = random.randint(0, q-1)
print(f"Verifier selects random challenge c = {c}")

# Cheating prover tries to fake a response without knowing x
fake_s = random.randint(0, q-1)
print(f"Cheating prover sends fake response fake_s = {fake_s}")

# Verifier checks
result = schnorr_verify(p, q, g, y, fake_t, c, fake_s)
print(f"Verification result: {result} (expected to fail)")
print()

=== Example 3: Simulating a Cheating Prover ===
Cheating prover selects random fake_r = 8 and computes fake_t = 78
Verifier selects random challenge c = 3
Cheating prover sends fake response fake_s = 4
Verification result: False (expected to fail)

